# Artificial Neural Network

### Importing Dependencies

In [32]:
import torch
import torchvision
import torch.nn as nn # contains all building blocks to create neural networks
import torch.nn.functional as F # contains all functions without parameters 
import torch.optim as optim # contains optimizers such as SGD, Adam, etc.
from torch.utils.data import DataLoader # helps with managing datasets by creating batches
import torchvision.datasets as datasets # contains standard datasets such as MNIST, CIFAR10, etc.
import torchvision.transforms as transforms # contains transformations to apply to datasets


### Creating a Simple Neural Network Model

In [16]:
class NN_Model(nn.Module):
    def __init__(self,input_size, num_classes):
        super(NN_Model, self).__init__()
        self.fc1 = nn.Linear(input_size, 100) # input layer
        self.fc2 = nn.Linear(100, 50) # hidden layer
        self.fc3 = nn.Linear(50, num_classes) # output layer

    # nn.Module requires to implement the forward method to compute forward pass
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # last layer so automatically assigns softmax

        return x

### Set Device

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### HyperParameters

In [18]:
input_size = 784 # for MNIST data of 28*28, unrolled to get 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
epoch = 5

### Loading Data

In [19]:
train_dataset = datasets.MNIST(root='dataset/',train=True,transform=transforms.ToTensor(),download=True,)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(root='dataset/',train=False,transform=transforms.ToTensor(),download=True,)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


### Model object

In [20]:
model = NN_Model(input_size=input_size, num_classes=num_classes).to(device)

### Loss and Optimizer

In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

### Training Loop


In [22]:
for epoch_i in range(epoch):
    losses = []
    accuracies = []
    print("Epoch:",epoch_i+1)
    for i,(image,label) in enumerate(train_dataloader):
        image = image.to(device)
        label = label.to(device)

        # Getting to proper shape
        image = image.reshape(image.shape[0],-1)

        # 5 Training steps
        logits = model(image)
        loss = criterion(logits, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        accuracies.append(label.eq(logits.detach().argmax(dim=-1)).float().mean())
    
    print(f"Epoch: {epoch_i+1} Train Loss: {torch.tensor(losses).mean():.2f}",end=' \t')
    print(f"Epoch: {epoch_i+1} Train Accuracy: {torch.tensor(accuracies).mean():.2f}")

Epoch: 1
Epoch: 1 Train Loss: 0.36 Epoch: 1 Train Accuracy: 0.89
Epoch: 2
Epoch: 2 Train Loss: 0.16 Epoch: 2 Train Accuracy: 0.95
Epoch: 3
Epoch: 3 Train Loss: 0.11 Epoch: 3 Train Accuracy: 0.97
Epoch: 4
Epoch: 4 Train Loss: 0.08 Epoch: 4 Train Accuracy: 0.97
Epoch: 5
Epoch: 5 Train Loss: 0.07 Epoch: 5 Train Accuracy: 0.98


In [29]:
def check_accuracy(data_loader, model):

    if data_loader.dataset.train:
        print("Checking accuracy on training data")
    else:
        print("Checking accuracy on test data")

    num_samples=[]
    num_correct=[]
    model.eval() # sets model in evaluation mode, by default it is in training mode

    with torch.no_grad():

        for i,(image,label) in enumerate(data_loader):
            image = image.to(device)
            label=label.to(device)

            image = image.reshape(image.shape[0],-1)
            
            # Testing Loop
            logits = model(image)
            
            _, preds = logits.max(1) # returns the index of the maximum value in a tensor along a given axis (1 is the axis here) 
            # if logits = [0.2, 0.5, 0.3,........] then it will return 1 as the index of the maximum value is 0.5 showing digit predicted is 1

            num_correct.append((preds == label).sum())
            num_samples.append(preds.size(0))
            accuracy = float(sum(num_correct)/sum(num_samples)*100)

        print(f"Accuracy = {accuracy:.2f}")
    model.train() # sets model back to training mode

    # return accuracy 
        

In [30]:
check_accuracy(test_dataloader,model)

Checking accuracy on test data
Accuracy = 97.16
